QA BOOT

In [35]:

import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:



def question_answer(question, reference):
    """
    """
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

    # Load the BERT model for question answering
    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    # Tokenize the input question and reference text
    reference_tokens = tokenizer.tokenize(reference)
    reference_tokens += ['[SEP]']  # Add the [SEP] token to the end of the reference text
    reference_tokens_ids = tokenizer.convert_tokens_to_ids(reference_tokens)

    question_tokens = tokenizer.tokenize(question)
    question_tokens = ['[CLS]'] + question_tokens + ['[SEP]']
    question_tokens_ids = tokenizer.convert_tokens_to_ids(question_tokens)
    question_tokens_ids = tokenizer.convert_tokens_to_ids(question_tokens)
    input_ids = question_tokens_ids + reference_tokens_ids
    input_mask = [1] * len(input_ids)
    input_types = [0] * len(question_tokens) + [1] * len(reference_tokens)
    input_ids, input_mask, input_types = map(
        lambda t: tf.expand_dims(
            tf.convert_to_tensor(t, dtype=tf.int32), 0), (input_ids, input_mask, input_types))
    outputs = model([input_ids, input_mask, input_types])
    short_start = tf.argmax(outputs[0][0][1:-1]) + 1
    short_end = tf.argmax(outputs[1][0][1:-1]) + 1
    tokens = question_tokens + reference_tokens
    answer_tokens = tokens[short_start: short_end + 1]
    if len(answer_tokens) is 0:
        answer = None
    else:
        answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

<>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-37-4d98f6ab0eca>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(answer_tokens) is 0:


In [38]:


with open('/content/drive/MyDrive/dataset/ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

print(question_answer('When are PLDs?', reference))

on - site days from 9 : 00 am to 3 : 00 pm


In [39]:

quit = ['exit', 'quit', 'goodbye', 'bye']

while True:
    question = input('Q: ').lower()
    if question in quit:
        print('A: Goodbye')
        break
    print('A:')


Q: Hello
A:
Q:  How are you?
A:
Q: BYE
A: Goodbye


In [40]:

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Load the BERT model for question answering
model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")


def answer_loop(reference):
    """ Tokenize the input question and reference text """
    reference_tokens = tokenizer.tokenize(reference)
    reference_tokens += ['[SEP]']  # Add the [SEP] token to the end of the reference text
    reference_tokens_ids = tokenizer.convert_tokens_to_ids(reference_tokens)

    def get_answer(question):
      """
      """
      question_tokens = tokenizer.tokenize(question)
      question_tokens = ['[CLS]'] + question_tokens + ['[SEP]']
      question_tokens_ids = tokenizer.convert_tokens_to_ids(question_tokens)
      question_tokens_ids = tokenizer.convert_tokens_to_ids(question_tokens)
      input_ids = question_tokens_ids + reference_tokens_ids
      input_mask = [1] * len(input_ids)
      input_types = [0] * len(question_tokens) + [1] * len(reference_tokens)
      input_ids, input_mask, input_types = map(
          lambda t: tf.expand_dims(
              tf.convert_to_tensor(t, dtype=tf.int32), 0), (input_ids, input_mask, input_types))
      outputs = model([input_ids, input_mask, input_types])
      short_start = tf.argmax(outputs[0][0][1:-1]) + 1
      short_end = tf.argmax(outputs[1][0][1:-1]) + 1
      tokens = question_tokens + reference_tokens
      answer_tokens = tokens[short_start: short_end + 1]
      if len(answer_tokens) is 0:
          answer = None
      else:
          answer = tokenizer.convert_tokens_to_string(answer_tokens)
      return answer

    quit = ['exit', 'quit', 'goodbye', 'bye']
    while True:
        question = input('Q: ')
        if question.lower() in quit:
            print('A: Goodbye')
            break
        answer = get_answer(question)
        if answer is None:
            print('A: Sorry, I do not understand your question.')
        else:
            print('A:', answer)

<>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-40-8f28eb9dc2b7>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(answer_tokens) is 0:


In [41]:

with open('/content/drive/MyDrive/dataset/ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

answer_loop(reference)

Q:  When are PLDs?
A: on - site days from 9 : 00 am to 3 : 00 pm
Q: What are Mock Interviews?
A: Sorry, I do not understand your question.
Q:  What does PLD stand for?
A: peer learning days
Q: EXIT
A: Goodbye


In [42]:

import os
import numpy as np

def semantic_search(corpus_path, sentence):
    """

    - corpus_path is the path to the corpus of reference
        documents on which to perform semantic search
    - sentence is the sentence from which to perform semantic search
    Returns: the reference text of the document most similar to sentence
    """
    reference = []
    reference.append(sentence)
    dirs = os.listdir(corpus_path)
    for file in dirs:
        if not file.endswith('.md'):
            continue
        with open(corpus_path+'/'+file, 'r', encoding='utf-8') as f:
            reference.append(f.read())
    e = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
    embeddings = e(reference)
    corr = np.inner(embeddings, embeddings)
    close = np.argmax(corr[0, 1:])
    return reference[close + 1]



In [43]:
print(semantic_search('/content/drive/MyDrive/dataset/ZendeskArticles', 'When are PLDs?'))

PLD Overview
Peer Learning Days (PLDs) are a time for you and your peers to ensure that each of you understands the concepts you've encountered in your projects, as well as a time for everyone to collectively grow in technical, professional, and soft skills. During PLD, you will collaboratively review prior projects with a group of cohort peers.
PLD Basics
PLDs are mandatory on-site days from 9:00 AM to 3:00 PM. If you cannot be present or on time, you must use a PTO. 
No laptops, tablets, or screens are allowed until all tasks have been whiteboarded and understood by the entirety of your group. This time is for whiteboarding, dialogue, and active peer collaboration. After this, you may return to computers with each other to pair or group program. 
Peer Learning Days are not about sharing solutions. This doesn't empower peers with the ability to solve problems themselves! Peer learning is when you share your thought process, whether through conversation, whiteboarding, debugging, or li

In [44]:
def qa(coprus_path):
  w = ['exit', 'quit', 'goodbye', 'bye']
  while 1:
      question = input('Q: ')
      if question.lower() in w:
          print('A: Goodbye')
          break
      else:
        reference = semantic_search(coprus_path, question)
        answer = question_answer(question, reference)
        if answer is None or answer == '':
            print('A: Sorry, I do not understand your question.')
        else:
            print('A: {}'.format(answer))

In [46]:
qa('/content/drive/MyDrive/dataset/ZendeskArticles')

Q: When are PLDs?
A: on - site days from 9 : 00 am to 3 : 00 pm
Q: What are Mock Interviews?
A: help you train for technical interviews
Q: What does PLD stand for?
A: peer learning days
Q: GOODBYE
A: Goodbye
